In [1]:
# importing pandas for reading and clearing data 
import pandas as pd
# reading the data set 
dataset2 = pd.read_excel("Input.xlsx")

In [2]:
# imported request module for send request to URL 
import requests
# imported BeautifulSoup for web sraping 
from bs4 import BeautifulSoup
# Loop through each URL in the dataset and send a GET request
for index, row in dataset2.iterrows():
    url = row['URL']
    try:
        response = requests.get(url, timeout=15)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            title = soup.title.string if soup.title else 'No Title'
            content = ' '.join([p.get_text() for p in soup.find_all('div', class_="td-post-content")])
            # Store the title and content as strings
            dataset2.at[index, 'Title'] = str(title)
            dataset2.at[index, 'Content'] = str(content)
    except requests.exceptions.RequestException as e:
        print(f"Failed to process URL {url}: {e}")

C:\Users\Bavas\AppData\Local\Temp\ipykernel_2804\2058058538.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Rise of telemedicine and its Impact on Livelihood by 2040 - Blackcoffer Insights' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  dataset2.at[index, 'Title'] = str(title)
C:\Users\Bavas\AppData\Local\Temp\ipykernel_2804\2058058538.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '
Telemedicine, the use of technology to diagnose and treat patients remotely, has been rising in recent years. With the advent of high-speed internet and improved video conferencing tools, healthcare providers are increasingly turning to telemedicine to provide care to patients in remote or underserved areas.Telemedicine, using technology to provide healthcare services remotely, has recently

Failed to process URL https://insights.blackcoffer.com/rise-of-e-health-and-its-impact-on-humans-by-the-year-2030/: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Failed to process URL https://insights.blackcoffer.com/rise-of-telemedicine-and-its-impact-on-livelihood-by-2040-2-2/: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Failed to process URL https://insights.blackcoffer.com/oil-prices-by-the-year-2040-and-how-it-will-impact-the-world-economy/: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Failed to process URL https://insights.blackcoffer.com/will-ai-replace-us-or-work-with-us/: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [3]:
# Reading Stopwords provided in text files as well as tokenizing it
import os
directory = 'StopWords/'
contents = []
for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        with open(os.path.join(directory, filename), 'r', encoding='latin-1') as file:
            contents.append(file.read())
combined_content = "".join(contents)
custom_stopwords = combined_content.split('\n')

In [4]:
import os
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
# Download necessary NLTK data
nltk.download('punkt_tab')
# Directory containing your text files
directory = 'MasterDictionary/'
# Initialize lists
positive_words = []
negative_words = []
# Read files and classify words
for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        filepath = os.path.join(directory, filename)
        with open(filepath, 'r', encoding='utf-8', errors='ignore') as file:
            lines = file.readlines()
            for line in lines:
                tokens = word_tokenize(line.strip())  # Tokenizing the line with NLTK
                for word in tokens:
                    # Assuming positive and negative words are labeled within the file content
                    if "positive" in filename.lower():
                        positive_words.append(word)
                    elif "negative" in filename.lower():
                        negative_words.append(word)

# Create a dictionary
sentiment_dict = {
    "positive": positive_words,
    "negative": negative_words
}

[nltk_data] Downloading package punkt_tab to
[nltk_data]     F:\NLP_Project_linkdin_profile\env\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [5]:
# If we not get data we need to fill nan there
dataset2['Content'] = dataset2['Content'].fillna("")

In [6]:
import pandas as pd
import nltk
import string
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

# Download necessary NLTK packages
nltk.download('punkt')
nltk.download('stopwords')


# List of personal pronouns to check for
personal_pronouns = {"i", "we", "my", "ours", "us"}

# Function to count syllables in a word
def count_syllables(word):
    word = word.lower().strip(string.punctuation)
    syllables = 0
    vowels = "aeiouy"
    if word and word[0] in vowels:
        syllables += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            syllables += 1
    if word.endswith(("es", "ed")):
        syllables -= 1
    if syllables < 1:
        syllables = 1
    return syllables

# Calculate readability and sentiment metrics
def calculate_metrics(text):
    stop_words = set(stopwords.words('english')).union(set(custom_stopwords))
    
    # Tokenize sentences and words
    sentences = sent_tokenize(text)
    words = word_tokenize(text)
    words_no_stop = [word for word in words if word.lower() not in stop_words]
    words_no_stop = [word.strip(string.punctuation) for word in words_no_stop]
    
    # Calculate word counts
    num_words = len(words_no_stop)
    num_sentences = len(sentences)
    num_complex_words = sum(1 for word in words_no_stop if count_syllables(word) > 2)
    total_words = len(words_no_stop)  # After stopword removal
    
    # Calculate syllables per word and avg word length
    syllables_per_word = sum(count_syllables(word) for word in words_no_stop) / total_words if total_words > 0 else 0
    avg_word_length = sum(len(word) for word in words_no_stop) / total_words if total_words > 0 else 0

    # Calculate personal pronoun count
    personal_pronoun_count = sum(1 for word in words if word.lower() in personal_pronouns)

    # Calculate sentiment scores
    positive_score = sum(1 for word in words_no_stop if word.lower() in positive_words)
    negative_score = sum(1 for word in words_no_stop if word.lower() in negative_words)
    
    # Calculate Polarity and Subjectivity
    polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
    subjectivity_score = (positive_score + negative_score) / (total_words + 0.000001)
    
    # Calculate readability metrics
    avg_sentence_length = num_words / num_sentences if num_sentences > 0 else 0
    percent_complex_words = num_complex_words / num_words if num_words > 0 else 0
    fog_index = 0.4 * (avg_sentence_length + percent_complex_words)
    
    return (positive_score, negative_score, polarity_score, subjectivity_score, avg_sentence_length, 
            percent_complex_words, fog_index, avg_sentence_length, num_complex_words, total_words, 
            syllables_per_word, personal_pronoun_count, avg_word_length)

# Apply the function to the dataset
dataset2['Positive_Score'], dataset2['Negative_Score'], dataset2['Polarity_Score'], dataset2['Subjectivity_Score'], \
dataset2['Avg_Sentence_Length'], dataset2['Percent_Complex_Words'], dataset2['Fog_Index'], \
dataset2['Avg_Number_Words_Per_Sentence'], dataset2['Complex_Word_Count'], dataset2['Word_Count'], \
dataset2['Syllable_Per_Word'], dataset2['Personal_Pronouns'], dataset2['Avg_Word_Length'] = \
    zip(*dataset2['Content'].apply(calculate_metrics))

# Print the DataFrame to check the result
print(dataset2)


[nltk_data] Downloading package punkt to
[nltk_data]     F:\NLP_Project_linkdin_profile\env\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     F:\NLP_Project_linkdin_profile\env\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


      URL_ID                                                URL  \
0      123.0  https://insights.blackcoffer.com/rise-of-telem...   
1      321.0  https://insights.blackcoffer.com/rise-of-e-hea...   
2     2345.0  https://insights.blackcoffer.com/rise-of-e-hea...   
3     4321.0  https://insights.blackcoffer.com/rise-of-telem...   
4      432.0  https://insights.blackcoffer.com/rise-of-telem...   
..       ...                                                ...   
109  50921.0  https://insights.blackcoffer.com/coronavirus-i...   
110  51382.8  https://insights.blackcoffer.com/coronavirus-i...   
111  51844.6  https://insights.blackcoffer.com/what-are-the-...   
112  52306.4  https://insights.blackcoffer.com/marketing-dri...   
113  52768.2  https://insights.blackcoffer.com/continued-dem...   

                                                 Title  \
0    Rise of telemedicine and its Impact on Livelih...   
1                                                  NaN   
2    Rise of e-health

In [7]:
dataset2

,URL_ID,URL,Title,Content,Positive_Score,Negative_Score,Polarity_Score,Subjectivity_Score,Avg_Sentence_Length,Percent_Complex_Words,Fog_Index,Avg_Number_Words_Per_Sentence,Complex_Word_Count,Word_Count,Syllable_Per_Word,Personal_Pronouns,Avg_Word_Length
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,Rise of telemedicine and its Impact on Livelih...,"\nTelemedicine, the use of technology to diagn...",76,22,0.551020,0.098000,17.857143,0.439000,7.318457,17.857143,439,1000,2.558000,2,6.788000
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,NaN,,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0.000000,0,0.000000
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,Rise of e-health and its impact on humans by t...,\n2020 was the year the world was ravaged by t...,21,27,-0.125000,0.071535,10.650794,0.321908,4.389081,10.650794,216,671,2.138599,3,6.041729
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,Rise of telemedicine and its Impact on Livelih...,"\n“More gains on quality, affordability and ac...",34,27,0.114754,0.079016,16.782609,0.376943,6.863821,16.782609,291,772,2.257772,7,6.415803
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,NaN,,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0.000000,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,50921.0,https://insights.blackcoffer.com/coronavirus-i...,Coronavirus: Impact on the Hospitality Industr...,\nBefore jumping on the topic I would like to ...,10,27,-0.459459,0.074297,19.153846,0.285141,7.775595,19.153846,142,498,2.150602,1,6.070281
110,51382.8,https://insights.blackcoffer.com/coronavirus-i...,Coronavirus impact on energy markets - Blackco...,\nAs the coronavirus spreads around the world ...,23,58,-0.432099,0.073636,100.000000,0.280000,40.112000,100.000000,308,1100,2.092727,3,5.990000
111,51844.6,https://insights.blackcoffer.com/what-are-the-...,impacts of COVID-19 on the world of work - Wha...,"\nFrom Alibaba to Ping An and Google to Ford, ...",86,33,0.445378,0.106726,16.397059,0.330045,6.690841,16.397059,368,1115,2.126457,0,5.943498
112,52306.4,https://insights.blackcoffer.com/marketing-dri...,Marketing Drives Results With A Focus On Probl...,"\nWhen\nthe British ruled India, many Indians\...",28,21,0.142857,0.057310,14.741379,0.286550,6.011172,14.741379,245,855,2.086550,8,5.610526


In [8]:
output_path = "text_analysis_output.xlsx"  
dataset2.to_excel(output_path, index=False)